In [4]:
from REINFORCE import Reinforce
%matplotlib inline
import gymnasium as gym
import highway_env
from matplotlib import pyplot as plt

# Controls:
CONTINUOUS_ACTIONS = False
TESTING = False
EPOCHS = 150
EPISODES = 15
STEP_W = 1e-3
STEP_T = 0.006
DISCOUNT = 0.85
configs = f"{"TEST" if TESTING else "DRIVE"}_{"CON" if CONTINUOUS_ACTIONS else "DIS"}_{EPOCHS}_{EPISODES}_{STEP_W}_{STEP_T}_{DISCOUNT}"
###########

if CONTINUOUS_ACTIONS and TESTING:
    env = gym.make("Pendulum-v1", render_mode="rgb_array")
elif CONTINUOUS_ACTIONS and not TESTING:
    env = gym.make('highway-fast-v0', config={"action": {"type": "ContinuousAction"}}, render_mode='rgb_array')
elif not CONTINUOUS_ACTIONS and TESTING:
    env = gym.make("CartPole-v1", render_mode="rgb_array")
elif not CONTINUOUS_ACTIONS and not TESTING:
    env = gym.make('highway-fast-v0', render_mode='rgb_array')
else:
    print("invalid controls, aborting...")
    quit()

urs_lst = []
ars_lst = []

model = Reinforce(env, epochs=EPOCHS, episodes_per_epoch=EPISODES, step_size_w=STEP_W, step_size_theta=STEP_T, discount=DISCOUNT)
ur, ar = model.learn(verbose=True)
urs_lst.append(ur)
ars_lst.append(ar)
env.reset()
print("Training complete.")

/Users/jonah/Desktop/ReinforcementLearning/.venv/lib/python3.12/site-packages/pygame/pkgdata.py:25: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  from pkg_resources import resource_stream, resource_exists


Epoch: 1, Average Reward: 9.23
Epoch: 2, Average Reward: 7.88
Epoch: 3, Average Reward: 6.27
Epoch: 4, Average Reward: 9.14


KeyboardInterrupt: 

In [5]:
plt.plot(ars_lst[0])
plt.title(f"REINFORCE: Average Rewards vs Epochs")
plt.xlabel("Epoch")
plt.ylabel(f"Average Reward of {EPISODES} Episode(s)")
plt.show()

IndexError: list index out of range

In [ ]:
from gymnasium.wrappers import RecordVideo
import datetime
dt = datetime.datetime.now()
# Define class to record test output runs
env = gym.make('highway-v0', config={"duration":120}, render_mode='rgb_array')
env = RecordVideo(env, video_folder="../../videos/", episode_trigger=lambda e: True, name_prefix=f"vpg_{configs}")
# Evaluation loop, deterministically stepping in our environment to test our learned actor policy
for episode in range(5):
    obs, info = env.reset()
    done = truncated = False
    score = 0
    while not (done or truncated):
        action, _ = model.predict(obs)
        next_obs, reward, done, truncated, info = env.step(action)
        obs = next_obs
        score += reward
    print(f"Recorded Episode {episode+1}: Score = {score:.2f}")
env.close()
print("Videos saved to 'videos/' folder.")

C:\Users\carey\AppData\Local\Programs\Python\Python313\Lib\site-packages\gymnasium\wrappers\rendering.py:293: UserWarning: WARN: Overwriting existing videos at C:\Users\carey\OneDrive\Desktop\Work\Uni\Year 3\Reinforcement Learning\Coursework1\videos folder (try specifying a different `video_folder` for the `RecordVideo` wrapper if this is not desired)
  logger.warn(


Recorded Episode 1: Score = 19.56
Recorded Episode 2: Score = 6.22
Recorded Episode 3: Score = 12.00
Recorded Episode 4: Score = 1.67
Recorded Episode 5: Score = 16.80
Videos saved to 'videos/' folder.


In [ ]:
import numpy as np
import os
path = f"save_data_{configs}"
os.makedirs(path, exist_ok=True)
rewards_path = os.path.join(path, "reinforce_average_rewards.csv")
with open(rewards_path, "w") as f:
    pass
np.savetxt(rewards_path,
        np.column_stack((np.arange(len(ar)), ar)),
        delimiter=",",
        header="episode,reward",
        comments="")
rewards_path = os.path.join(path, "reinforce_undiscounted_rewards.csv")
with open(rewards_path, "w") as f:
    pass
np.savetxt(rewards_path,
        np.column_stack((np.arange(len(ur)), ur)),
        delimiter=",",
        header="episode,reward",
        comments="")